In [1]:
import cv2
import matplotlib.pyplot as plt
import importlib
import os
from ipywidgets import interact
from ipywidgets import interact_manual
from IPython.display import display
from ipywidgets import widgets

In [2]:
import ipynb.fs.full.wspFFA as wspFirefly
import ipynb.fs.full.wspCS as wspCuckooSearch
import ipynb.fs.full.wspKH as wspKrillHerd
import ipynb.fs.full.wspEHO as wspElephantHerding
import ipynb.fs.full.wspABC as wspArtificialBeeColony

import ipynb.fs.full.plotGraph as plotGraph
import ipynb.fs.full.wspMultithreshold as wspMultithreshold
import ipynb.fs.full.dicomHandler as dicomHandler

importlib.reload(wspFirefly)
importlib.reload(wspCuckooSearch)
importlib.reload(wspKrillHerd)
importlib.reload(wspElephantHerding)
importlib.reload(wspArtificialBeeColony)

importlib.reload(plotGraph)
importlib.reload(wspMultithreshold)
importlib.reload(dicomHandler)

<module 'ipynb.fs.full.dicomHandler' (/mnt/6884ACCB84AC9CD6/OneDrive - FEI/Aulas/Graduacao/8 - Semestre/CC8421 - TRABALHO FINAL DE CURSO II/image-segmentation/dicomHandler.ipynb)>

In [3]:
def readImage(img_path):
    image = cv2.imread(img_path)
    img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    return img

In [4]:
bio_algorithms = ['FFA','KH', 'CS', 'ABC', 'EHO']

In [5]:
folder_dir = "images"

data_dir = "../Database/UsevillaBone/original/"
gt_dir = "../Database/UsevillaBone/segmented/"

all_images = []
gt_images = []

for fatia in os.listdir(data_dir):
    all_images.append(data_dir+fatia)
    gt_images.append(gt_dir+fatia)
all_images = ["../Database/UsevillaBone/original/CASE_20_1.npy", "../Database/UsevillaBone/original/CASE_10_2.npy", "../Database/UsevillaBone/original/CASE_22_8.npy", "../Database/UsevillaBone/original/CASE_4_4.npy", "../Database/UsevillaBone/original/CASE_9_3.npy", "../Database/UsevillaBone/original/CASE_7_9.npy"]
gt_images = ["../Database/UsevillaBone/segmented/CASE_20_1.npy", "../Database/UsevillaBone/segmented/CASE_10_2.npy", "../Database/UsevillaBone/segmented/CASE_22_8.npy", "../Database/UsevillaBone/segmented/CASE_4_4.npy", "../Database/UsevillaBone/segmented/CASE_9_3.npy", "../Database/UsevillaBone/segmented/CASE_7_9.npy"]
#for file in os.listdir(folder_dir):
#    if file.endswith('.png') or file.endswith('.jpg'):
#       all_images.append(file)

In [6]:
import numpy as np

In [7]:
def main(image, algorithm, dimension, q, b, kernel):
    pixel_array = dicomHandler.read_npy_image(image)
    
    blur = cv2.GaussianBlur(pixel_array, (kernel,kernel), 0)

    print("Original:", pixel_array.min(), pixel_array.max())
    transformed_pixel_array = dicomHandler.image_transformation(blur, b)
    print("Transformação:", transformed_pixel_array.min(), transformed_pixel_array.max())


    plotGraph.plot_n_images([pixel_array, blur, transformed_pixel_array, np.load(gt_images[all_images.index(image)])])

    original_image = pixel_array.copy()

    hu_image = dicomHandler.transform_npy_to_hu(transformed_pixel_array)

    hist, bin_edges, best_thresholds, img_thres = wspMultithreshold.wspMultithreshold(hu_image, algorithm, dimension, q)

    high_intensity = wspMultithreshold.get_high_intensity_pixels(img_thres)
    
    pixel_array = dicomHandler.transform_to_npy_pixel_array(high_intensity)

    hard_tissue = wspMultithreshold.get_largests_regions(pixel_array, original_image)
    
    plotGraph.plot_image_histogram(hu_image, hist, bin_edges)
    plotGraph.plot_image_histogram_threshold(img_thres, hist, bin_edges, best_thresholds)
    
    plotGraph.plot_n_images([high_intensity, hard_tissue, np.load(gt_images[all_images.index(image)])])

    #dicomHandler.save_npy(high_intensity, "teste.npy")

In [8]:
interact_manual(main, image=all_images, algorithm=bio_algorithms, 
    dimension=widgets.IntSlider(min=1, max=5, step=1, value=1),
    q=widgets.FloatSlider(min=-2, max=2, step=0.1, value=1),
    b=widgets.FloatSlider(min=1, max=50, step=0.1, value=1.5),
    kernel=widgets.IntSlider(min=1,max=10, step=1, value=3))

interactive(children=(Dropdown(description='image', options=('../Database/UsevillaBone/original/CASE_20_1.npy'…

<function __main__.main(image, algorithm, dimension, q, b, kernel)>